## Sproj Board Scheduler

An attempt to write a script that would read faculty and students availability for the boards week, and then schedule all boards, accounting for their target composition, and student availability.

Unfortunately some of the data used by this script (names) is sensitive, and had to be stored outside of git.

In [51]:
import pandas as pd
import time
import copy
import random
from IPython.display import HTML, display

In [52]:
hideNames = 1 # set 0 for troubleshooting, set 1 before githubbing

In [53]:
composition = pd.read_csv('../../sensitive_data/boards.csv')
students = pd.read_csv('../../sensitive_data/students.csv')
facultin = pd.read_csv('../../sensitive_data/faculty.csv')

In [54]:
# Create time grid
days = ['m','t','w','r','f']
dayName = {'m':'Mon','t':'Tue','w':'Wed','r':'Thu','f':'Fri'}

startTime = 9
endTime = 18
grid = [] # Array of tuples: day (as a letter), hour (as a 24h int number)
for d in days:
    for t in range(startTime,endTime):
        grid = grid + [(d,t)]
print(grid[:3])

facultin.time = facultin.time.apply(lambda x: max(x,startTime))
# Note that the script doesn't check times outside of start-end window, so all ins and outs should happen either
# within this window, or before the day starts (to add or remove an entire day)

[('m', 9), ('m', 10), ('m', 11)]


In [55]:
class Faculty:
    def __init__(self,name):
        self.name = name
        self.avail = []
        
    def __str__(self):
        return "%12s \t" % (self.name) + ''.join(['%d' % i for i in self.avail])
    
    def initAvail(self,grid,facultin):
        self.avail = [1]*len(grid)
        status = 1
        for ig in range(len(grid)):
            lin = facultin.loc[(facultin['name']==self.name) & 
                               (facultin['type']=='in') & 
                               (facultin['day']==grid[ig][0]) &
                               (facultin['time']==grid[ig][1])]
            if lin.shape[0]>0:
                status = 1
            lout = facultin.loc[(facultin['name']==self.name) & 
                               (facultin['type']=='out') & 
                               (facultin['day']==grid[ig][0]) &
                               (facultin['time']==grid[ig][1])]
            if lout.shape[0]>0:
                status = 0
            self.avail[ig] = status
            
    def updateAvail(self,grid,g,newVal=0):
        self.avail = [self.avail[i] if grid[i]!=g else newVal for i in range(len(self.avail))]
        
    def book(self,ig):
        self.avail[ig] = 0
        
        
faculty = []
for fn in composition['advisor'].append(composition['mem2']).append(composition['mem3']).unique():
    f = Faculty(fn)
    f.initAvail(grid,facultin)
    for d in days:
        f.updateAvail(grid,(d,12)) # Book lunch breaks
        if d=='w': # Wed is an advising day, so take it out completely
            for ih in range(1,25):
                f.updateAvail(grid,(d,ih),0)
    faculty.append(f)    

for f in faculty:
    print(f)

        Jude 	111011100111011111000000000000011111111011111
    Tibbetts 	000000000111011111000000000000000000000000000
   Robertson 	111011111111011111000000000111011111111011111
     Collins 	111011111111011111000000000111011111111011111
     Keesing 	111011111111011111000000000000000000000000000
   Khakhalin 	111011111111011111000000000111011111111011111
      Perron 	111011111111011111000000000111011111111011111
     Bennett 	000000000000000000000000000111011111111011111
      Dueker 	111011111111011111000000000111011111111011111
        Jain 	111011111111011111000000000111011111111011111
      Scalzo 	111011111111011111000000000111011111111011111


In [56]:
class Board:
    '''Board object'''
    
    def __init__(self,stuff): # Creator
        self.student = stuff['first'] + ' ' + stuff['last']
        if hideNames:
            self.student = ''.join(random.sample(self.student.lower(),len(self.student)))
        self.members = list(stuff[['advisor','mem2','mem3']])
        self.email = stuff['email']
        self.type = stuff['type']
        self.avail = []  # Placeholder: availability grid
        self.time = []
        
    def __str__(self):
        s = "%25s" % (self.student) + '\t' 
        s += ' '.join([m[:4] for m in self.members]) + '\t' # Shortened version
        s += ''.join(['%d' % i for i in self.avail])
        return s
    
    def initTimes(self,grid,students):
        '''Sets boards initial options, based on students availability'''
        data = students.loc[students.email==self.email]
        self.avail = [1]*len(grid)
        for i in range(data.shape[0]):
            if data.iloc[i]['class'][:3]=='BIO':
                continue # Ignore bio classes
            for d in days:
                if not pd.isna(data.iloc[i][d]): # Class meets on this day
                    times = data.iloc[i]['time'].split('-') # String of two times
                    for i in range(2):
                        if times[i][0]==' ':
                            times[i] = times[i][1:] # Remove leading spaces
                    tstart = time.strptime(times[0], "%I:%M %p")
                    tend   = time.strptime(times[1], "%I:%M %p")
                    for ig in range(len(grid)):
                        g = grid[ig]
                        if g[0]==d: # If this day
                            if g[1]>=tstart.tm_hour and g[1]<=tend.tm_hour: # If classes during this hour
                                self.avail[ig] = 0
                                
    def refreshFac(self,faculty):
        '''Filters boards based on current faculty availability'''
        for f in faculty:
            for facname in self.members:
                if f.name==facname:
                    self.avail = [self.avail[i]*f.avail[i] for i in range(len(self.avail))]
          
    
boards = []
for ib in range(len(composition)):
    b = Board(composition.loc[ib,])
    b.initTimes(grid,students)
    b.refreshFac(faculty)
    boards.append(b)

# Rearrange from those that are harder to schedule to those that are easier
niceness = [sum(b.avail) for b in boards]
ind = [i for _,i in sorted(zip(niceness,range(len(niceness))))]
ind = [j for (i,j) in sorted(zip(ind,range(len(ind))))] # Resolve duplicates
boards = [boards[i] for i in ind]

#for b in boards:
#    print(b)

In [57]:
# Backup, just so that the cell below could be rerun without ruining the data
bb = copy.deepcopy(boards)
bf = copy.deepcopy(faculty)

In [58]:
boards = copy.deepcopy(bb)
faculty = copy.deepcopy(bf)

for b in boards:
    b.refreshFac(faculty)
    temp = [i for i in range(len(b.avail)) if (b.avail[i]==1)]
    if len(temp)==0:
        print('Cannot solve the puzzle for this board:')
        print(b)
        break
    ig = min(temp)
    b.time = grid[ig]
    b.avail[ig] = 0
    for f in faculty:
        if f.name in b.members:
            f.book(ig)
    print('%s\t%s\t' % (dayName[grid[ig][0]],grid[ig][1]), end='')
    print(b)
    
#for b in boards:
#    print(b)

Thu	14	          aoraidehleccr m	Khak Jude Benn	000000000000000000000000000000000111111011111
Mon	9	                ads neyio	Jude Coll Kees	011011100111011111000000000000000000000000000
Mon	10	         ot oylteanrinlsp	Duek Kees Perr	001000011111011100000000000000000000000000000
Mon	11	         blehahtemiazt os	Kees Coll Khak	000011100100011111000000000000000000000000000
Thu	13	               anr satqia	Khak Jude Benn	000000000000000000000000000000000111111011111
Mon	16	               r neantbby	Coll Kees Perr	000000001110001111000000000000000000000000000
Mon	9	             a mydnndazro	Robe Perr Duek	001011101111011111000000000111011111111011111
Mon	13	              ete kopacor	Kees Coll Robe	000001101111011100000000000000000000000000000
Mon	11	            a nymbtbgarah	Robe Perr Duek	000000001111000111000000000111011100111000001
Thu	9	              ny mavaenoc	Perr Benn Khak	000000000000000000000000000011000111111011111
Thu	10	            ncdnicuuri bo	Benn Perr Khak	000000000000000

In [59]:
ind = [min([i for i in range(len(grid)) if grid[i]==b.time]) for b in boards]
ind = [j for (i,j) in sorted(zip(ind,range(len(ind))))]
boards = [boards[i] for i in ind]

def printboard(b,mode="none"):
    if mode=="none":
        print("%22s\t%12s\t%s\t%d\t" % (b.student,b.type,dayName[b.time[0]],b.time[1]),end='')
        for fn in b.members:
            print("%10s\t" % (fn),end='')
        print()
    else:
        s = ''
        s += "<tr><td>%s</td><td>%s</td><td>%s %d</td><td>" % (b.student,b.type,dayName[b.time[0]],b.time[1])
        for fn in b.members:
            s += "%s " % (fn)
        s += "</td></tr>"
        display(HTML(s))

for b in boards:
    printboard(b)

             ads neyio	       final	Mon	9	      Jude	   Collins	   Keesing	
          a mydnndazro	      midway	Mon	9	 Robertson	    Perron	    Dueker	
      ot oylteanrinlsp	      midway	Mon	10	    Dueker	   Keesing	    Perron	
      blehahtemiazt os	      midway	Mon	11	   Keesing	   Collins	 Khakhalin	
         a nymbtbgarah	      midway	Mon	11	 Robertson	    Perron	    Dueker	
           ete kopacor	       final	Mon	13	   Keesing	   Collins	 Robertson	
   lmlaatsename atcraa	      midway	Mon	13	    Perron	      Jude	    Scalzo	
         eiya lagercmn	  moderation	Mon	14	   Keesing	      Jude	 Khakhalin	
           zuaifa endr	       final	Mon	15	 Robertson	      Jude	   Keesing	
            r neantbby	      midway	Mon	16	   Collins	   Keesing	    Perron	
        ahehicark rhnn	      midway	Mon	17	    Perron	    Dueker	      Jain	
          wo llsabmari	       final	Mon	17	   Collins	 Robertson	   Keesing	
            k amenrseo	  moderation	Tue	9	   Keesing	      Jude	 Khakhalin	
  

In [60]:
#Sort by faculty:

for f in faculty:
    print()
    print(f.name)
    for b in boards:
        if f.name in b.members:
            printboard(b)


Jude
             ads neyio	       final	Mon	9	      Jude	   Collins	   Keesing	
   lmlaatsename atcraa	      midway	Mon	13	    Perron	      Jude	    Scalzo	
         eiya lagercmn	  moderation	Mon	14	   Keesing	      Jude	 Khakhalin	
           zuaifa endr	       final	Mon	15	 Robertson	      Jude	   Keesing	
            k amenrseo	  moderation	Tue	9	   Keesing	      Jude	 Khakhalin	
          jbnsiguloia 	      midway	Tue	10	      Jude	   Keesing	  Tibbetts	
          ledy tgneecn	      midway	Tue	11	      Jude	  Tibbetts	   Collins	
          lhehmucai le	      midway	Tue	13	      Jude	  Tibbetts	 Khakhalin	
 olnp-fairklipht ezoee	  moderation	Tue	14	   Keesing	      Jude	 Khakhalin	
      iuraagvlazeqesb 	       final	Tue	15	  Tibbetts	      Jude	 Khakhalin	
            anr satqia	      midway	Thu	13	 Khakhalin	      Jude	   Bennett	
       aoraidehleccr m	      midway	Thu	14	 Khakhalin	      Jude	   Bennett	
           etmlwijhse 	      midway	Thu	15	   Collins	      Jude	   Benn